In [5]:
from mpi4py import MPI
import pickle
import timeit
import msgpack
import sys
from test_pb2 import BunchOfTestDicts, TestDict, Pair

comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size
name = MPI.Get_processor_name()

shared = (rank+1) * 3

comm.send(shared, dest = (rank+1)%size)
data = comm.recv(source = (rank-1)%size)

print name
print "Rank:", rank
print "Recieved:", data, "which came from rank:", (rank-1)%size

def writePB():
    bOTD = BunchOfTestDicts()
    for thisDict in realStuff:
        tD = bOTD.dicts.add()
        for k, v in thisDict.items():
            pair = tD.pairs.add()
            pair.key = k
            pair.value = v

    with open('realstuff.pb', 'wb') as f:
        f.write(bOTD.SerializeToString())

def readPB():
    bOTD = BunchOfTestDicts()
    with open('realstuff.pb', 'rb') as f:
        bOTD.ParseFromString(f.read())
    thisDictList = [{thisPair.key: thisPair.value
                             for thisPair in thisBufferedDict.pairs}
                    for thisBufferedDict in bOTD.dicts]
    return thisDictList

def writeReadPB():
    bOTD = BunchOfTestDicts()
    for thisDict in realStuff:
        tD = bOTD.dicts.add()
        for k, v in thisDict.items():
            pair = tD.pairs.add()
            pair.key = k
            pair.value = v
    #serializedPB = bOTD.SerializeToString()
    #print 'serialized PB length: %s'%len(serializedPB)
    #newBOTD = BunchOfTestDicts()
    #newBOTD.ParseFromString(serializedPB)
    newBOTD = bOTD
    thisDictList = [{thisPair.key: thisPair.value
                             for thisPair in thisBufferedDict.pairs}
                    for thisBufferedDict in newBOTD.dicts]
    return thisDictList


with open('realstuff.pkl', 'rb') as f:
    realStuff = pickle.load(f)

setupStatement="""\
from __main__ import writePB, readPB, writeReadPB, realStuff
"""

#print ('write: %s' % timeit.timeit("writePB()", setup=setupStatement, number=1))
#print ('read:  %s' % timeit.timeit("readPB()", setup=setupStatement, number=1))
print ('writeRead:  %s' % timeit.timeit("writeReadPB()", setup=setupStatement, number=1))


def writeReadPkl():
    serializedPkl = pickle.dumps(realStuff, protocol=2)
    print ('pickle string length: %s'%len(serializedPkl))
    rslt = pickle.loads(serializedPkl)
    return rslt

setupStatement="""\
from __main__ import writeReadPkl, realStuff
"""

print ('writeRead:  %s' % timeit.timeit("writeReadPkl()", setup=setupStatement, number=1))

def writeReadMSG():
    serializedMSG = msgpack.dumps(realStuff)
    print ('MsgPack length: %s'%len(serializedMSG))
    rslt = msgpack.loads(serializedMSG)
    return rslt

setupStatement="""\
from __main__ import writeReadMSG
"""

print ('writeRead:  %s' % timeit.timeit("writeReadMSG()", setup=setupStatement, number=1))



theotherjc-SVE17127CXB
Rank: 0
Recieved: 3 which came from rank: 0
writeRead:  2.35731101036
pickle string length: 23798577
writeRead:  9.25399708748
MsgPack length: 16099141
writeRead:  0.196992874146
